# Logistic Regression Modelling

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

In [17]:
df = pd.read_excel('Cleaned_Date.xlsx')

In [18]:
X = df.drop('Response', axis=1).iloc[:,1:][['MntFishProducts', 'MntMeatProducts', 'MntFruits', 'MntSweetProducts', 'MntWines', 'MntGoldProds', 'Age', 'Income', 'Kidhome', 'Teenhome']]
y = df['Response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [20]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
preds = clf.predict(scaler.transform(X_test))

In [21]:
acc = accuracy_score(y_test, preds)
prec = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
auc = roc_auc_score(y_test, preds)

In [22]:
print("Accuracy: %.4f" % acc)
print("Precision: %.4f" % prec)
print("Recall: %.4f" % rec)
print("F1: %.4f" % f1)
print("AUC: %.4f" % auc)

Accuracy: 0.8474
Precision: 0.4400
Recall: 0.1122
F1: 0.1789
AUC: 0.5437
